In [ ]:
pip install tensorflow

In [ ]:
pip install transformers

In [ ]:
pip install bs4

In [ ]:
pip install requests

In [ ]:
pip install pandas

In [ ]:
pip install nltk

In [ ]:
pip install rouge_score

In [ ]:
pip install evaluate

In [ ]:
pip install bert_score

In [ ]:
pip install torch

In [ ]:
pip install numpy

In [ ]:
pip install meteor

In [ ]:
import requests
from bs4 import BeautifulSoup
import transformers
from transformers import pipeline
pipe = pipeline("summarization", model="facebook/bart-large-cnn")

In [ ]:
from evaluate import load
import evaluate

In [ ]:
import transformers
from transformers import BertTokenizer, BertForMaskedLM, BertModel
import bert_score
from bert_score import BERTScorer

In [ ]:
import re
import pandas as pd

In [ ]:
def get_data(url):
    """ Retrieves text surrounded by <p> tags from the url

        Args:
          url: A url passed in by the user, containing the text to be summarised

        Returns:
          data: An array of each paragraph from the url
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    data = soup.find_all('p')
    return data

In [ ]:
#text = str (text)
#text

In [ ]:
def clean_data(text):
  """ Cleans the data so it can be processed by the pipeline

  Args:
    text: raw data from the get_data() method

  Returns:
    paragraph: An array containing all the clean data, separated by paragraph
  """
  paragraph = []
  for i in text:
    paraString = str (i)
    match = re.match("<p>(.*)<\/p>", paraString)
    if match:
      groupMatch = re.match("^(?!.*<[^>]+>).*$", match.group(1))
      if groupMatch:
        paragraph.append(match.group(1))
  return paragraph

In [ ]:
def get_summary(text, maxLim, minLim, sample):
  """ Summarizes the text paragraph by paragraph

  Args:
    text: An array of clean text(s)
    max: An integer stating the maximum amount of words the summary should include
    min: An integer stating amount of words the summary should include
    sample: A boolean that influences how the model generates text

  Returns:
    summary: A dictionary full of key-value pairs ('summary_text': summarised paragraph)
  """
  summary = pipe(text, max_length=maxLim, min_length=minLim, do_sample=sample)
  return summary

In [ ]:
def get_dataframe(para, summ):
  """ Turns the original paragraphs and summarised paragraphs into a Data Frame

    Args:
      para: The original cleaned paragraphs
      summ: The summary dictionary

    Returns:
      textDF: Data Frame containing both values of para and summ
  """
  summaries = []
  for i in range(len(summ)):
    summaries.append(summ[i]['summary_text'])
  para = pd.DataFrame(para, columns=['Paragraph'])
  summaries = pd.DataFrame(summaries, columns=['Summary'])
  textDF = pd.concat([para, summaries], axis=1)
  pd.set_option('display.max_colwidth',3000)
  return textDF

In [ ]:
#text = get_data('https://www.forbes.com/sites/forbestechcouncil/2023/04/10/the-future-of-artificial-intelligence/?sh=6043be4ac496')
text = """
The tower is 324 metres(1,063 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction. the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world, a title it held for 41 year until the Chrysler Building in New York City was finished in 1930. It was the first structure to reach a height of 300 metres. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17ft). Excluding transmitters, the Eiffel Tower is the second tallest is second tallest free-standing structure in France after the Millau Viaduct.

Big Ben is the nickname of the Great Bell inside the Elizabeth Tower, a clock tower at the Palace for Westminster in London. It is one of the most famous landmarks and symbols of the United Kingdom and is parliamentary democracy. The tower was completed in 1859 and the bell was installed in the same year. The clock is very accurate and can be adjusted by adding or removing pennies from the pendulum. The tower is 96 metres tall and has four clock faces, each 7 metres in diameter. The bell weighs 13.7 tonnes and may have been named after Sir Benjamin Hall or Benjamin Caunt, a boxer. The tower was renamed in 2012 to honour Queen Elizabeth II on her Diamond Jubilee.

The Colosseum is a famous amphitheatre in Rome, Italy. It was built by the Flavian emperors between 72 and 80 AD, using stone, concrete, and tens of thousands of Jewish slaves. The Colosseum could seat about 5,000 spectators, who came to watch various events, such as gladiator fights, animal hunts and mock naval battles. The Colosseum had complex system of vaults, arches and trap doors, as well as an awning to provide shade. The Colosseum was named after a colossal statue of Emperor Nero that stood nearby. The Colosseum is a symbol of ancient Roman civilization and its culture of entertainment and violence. It is also a testament to the engineering and architectural skills of the Romans. Today, the Colosseum is a popular tourist attraction and a UNESCO World Heritage Site.
"""

In [ ]:
#cleanText = clean_data(text)
cleanText = text.split("\n")

In [ ]:
summary = get_summary(cleanText, 75, 25, False)

In [ ]:
summaryDF = get_dataframe(cleanText, summary)
summaryDF

In [ ]:
rouge = evaluate.load('rouge')
candidates = summaryDF['Summary']
references = summaryDF['Paragraph']
results = rouge.compute(predictions=candidates, references=references)
results

{'rouge1': 0.6788011807170491,
 'rouge2': 0.6571257725608859,
 'rougeL': 0.6004623391584141,
 'rougeLsum': 0.6042997616219998}

In [ ]:
import torch
import numpy as np

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

text1 = summaryDF['Paragraph'].tolist()
text2 = summaryDF['Summary'].tolist()

inputs1 = tokenizer(text1, return_tensors="pt", padding=True, truncation=True)
inputs2 = tokenizer(text2, return_tensors="pt", padding=True, truncation=True)

outputs1 = model(**inputs1)
outputs2 = model(**inputs2)

embeddings1 = outputs1.last_hidden_state.mean(dim=1).detach().numpy()
embeddings2 = outputs2.last_hidden_state.mean(dim=1).detach().numpy()

similarity = np.dot(embeddings1, embeddings2.T) / (np.linalg.norm(embeddings1) * np.linalg.norm(embeddings2))

results = "Similarity between the texts: {:.4f}".format(similarity[0][0])
results

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

'Similarity between the texts: 0.0602'

In [ ]:
meteor = evaluate.load('meteor')
predictions = summaryDF['Summary']
reference = summaryDF['Paragraph']
results = meteor.compute(predictions=predictions, references=reference)
#rounded_results = round(results['meteor'], 2
results

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


{'meteor': 0.5340975152142372}